# Importing stuff and declaring constants


In [ ]:
import os
import pandas as pd
import calendar
from datetime import datetime
# import matplotlib_inline
import matplotlib.pyplot as plt
import numpy as np

TOTAL_MONTHLY_HRS = 150
monthly_hours = {calendar.month_name[i]: TOTAL_MONTHLY_HRS for i in range(1, 13)}
monthly_hours = pd.Series(monthly_hours)
monthly_hours

# Reading Excel


In [ ]:
allocation_sheet_df = pd.read_excel(os.path.join(
    os.getcwd(), './Main.xlsx'), sheet_name='Allocation')
employee_sheet_df = pd.read_excel(os.path.join(
    os.getcwd(), './Main.xlsx'), sheet_name='Employees')
project_sheet_df = pd.read_excel(os.path.join(
    os.getcwd(), './Main.xlsx'), sheet_name='Projects')


# Filtering Data


In [ ]:
def get_month_number(month_name):
    months = {month.lower(): index for index, month in enumerate(
        calendar.month_name) if month}
    return months[month_name.lower()]


allocation_sheet_df['Month'] = [get_month_number(i) for i in allocation_sheet_df['Month']]
allocation_sheet_df.dropna(inplace=True)
allocation_sheet_df.drop_duplicates(inplace=True)
allocation_sheet_df


# Creating Employees Dictionary which has everything

This has all the data about that employee. Now this is a dictionary, and you can use this dictionary which has important data to then plot graphs


In [ ]:
employees = {}
# template
# employee = {
#     "name": "",
#     "position": "",
#     "projects": {'name': '', 'months' : months},
# }

for i in range(employee_sheet_df.__len__()):
    employee = {'name': employee_sheet_df.iloc[i].values[0],
                'branch': employee_sheet_df.iloc[i].values[1],
                'projects': {},
                }
    employees[employee_sheet_df.iloc[i].values[0]] = employee


newdf = []
for _, i in enumerate(employees):
    emp_df = allocation_sheet_df[i == allocation_sheet_df['Name of Employee']]
    # adding projects
    for j in range(len(emp_df)):
        employees[i]['projects'][emp_df.iloc[j]['Project Name']] = {}
        employees[i]['projects'][emp_df.iloc[j]['Project Name']] = {k: v for (k, v) in zip(
            [calendar.month_name[i] for i in range(1, 13)], [0 for i in range(1, 13)])}
    # assigning time for each project
    for j in range(len(emp_df)):
        employees[i]['projects'][emp_df.iloc[j]['Project Name']][calendar.month_name[emp_df['Month'].iloc[j]]] += \
            emp_df.iloc[j]['Number of Hours']



In [ ]:
employees


# Creating Employee df


In [ ]:
employee_df = pd.DataFrame(employees).transpose()
# employee_df[[True if 'Karim Services' in i.keys() else False for i in employee_df.projects ]]
employee_df


# Graphs


In [ ]:
plt.style.use('default')

In [ ]:
employee_df

In [ ]:
# so we want to plot 4 employees on a single horizontal bar chart. The legend must have the name of the projects that they are working on.
# the x axis must have the months and the y axis must have the names of the employees.
# the bar chart must have the number of hours that each employee is working on each project.
# The legend must have the names of only those projects that the 4 employees we are displaying are working on.
# So to get the legend we must first get our four employees. Then we must get the projects that they are working on.

EMPLOYEE_PER_GRAGH = 4
current_employees_df = employee_df.iloc[:EMPLOYEE_PER_GRAGH]
projects_list = []

# get the projects that the current employees are working on.

for i in current_employees_df.iterrows():
    projects_list.extend(i[1].projects.keys())

projects_list = list(set(projects_list))
projects_list.append('free_time')
category_colors_1 = plt.colormaps['RdYlGn'](np.linspace(0.15, 1, projects_list.__len__() - 1))

category_color_green = [[0.429527104959631, 0.7540945790080739, 0.3914733082901554, 1.0]]
colors_series = pd.Series(category_colors_1.tolist())
colors_series = colors_series.sample(frac=1)

# concat the green color to the end of the series
colors_series = pd.concat([colors_series, pd.Series(category_color_green)])

print(colors_series)
category_colors = pd.Series([i for i in colors_series], index=projects_list)
# print(projects_list)
print(category_colors)

these_months = [calendar.month_name[i] for i in range(datetime.now().month, datetime.now().month + 4)]

In [ ]:
# now well create dataframes for each person for each project. The project list only has the names of the projects that the four employees are working on.
employee_project_dataframes = {}
employee_project_dataframes_cumsum = {}
for i in current_employees_df.iterrows():
    current_employee_df = pd.DataFrame(
        index=projects_list, columns=these_months)
    for j in i[1].projects.keys():
        current_employee_df.loc[j] = i[1].projects[j]
    current_employee_df.fillna(0, inplace=True)
    current_employee_df.loc['free_time'] = monthly_hours - current_employee_df.sum(axis=0)
    # current_employee_df['Colors'] = [i for i in category_colors]
    # convert things in the dataframe to np arrays
    for j in current_employee_df.columns:
        current_employee_df[j] = np.array(current_employee_df[j])
    
            
    print(current_employee_df)
    employee_project_dataframes[i[0]] = current_employee_df
    employee_project_dataframes[i[0]] = employee_project_dataframes[i[0]][employee_project_dataframes[i[0]] .columns[::-1]]
    employee_project_dataframes_cumsum[i[0]] = current_employee_df.cumsum(axis=0)
    employee_project_dataframes_cumsum[i[0]] = employee_project_dataframes_cumsum[i[0]][employee_project_dataframes_cumsum[i[0]] .columns[::-1]]


In [ ]:
employee_project_dataframes['Ramesh R'][employee_project_dataframes['Ramesh R'].columns[::-1]]


In [ ]:
labels = list(reversed(these_months))
first_one_done = False
# making 4 subplots
fig, axes = plt.subplots(EMPLOYEE_PER_GRAGH, sharex=True, sharey=True, figsize=(15, 5))
plt.xlabel('Months')
plt.subplots_adjust(hspace=0.1)

# plt.tick_params(
#     axis='x',          # changes apply to the x-axis
#     which='both',      # both major and minor ticks are affected
#     bottom=False,      # ticks along the bottom edge are off
#     top=False,         # ticks along the top edge are off
#     labelbottom=False)  # labels along the bottom edge are off

# iterating through 4 subplots, each subplot is a different employee
for ax, emp_names in zip(axes, employee_project_dataframes_cumsum.keys()):

    # remove ticks on both axes for employees. 
    ax.tick_params(axis='x', which='both', bottom=False, 
                   top=False, labelbottom=False)
    # ax.tick_params(axis='y', which='both', left=False,
    #                labelleft=False)
    # ax.invert_yaxis()
    print(emp_names, '---------------------------------')
    ax.set_ylabel(emp_names, rotation=0, labelpad=80)

    # iterating through each project for that employee, there will be a few projects, and the y axis would have the months
    for i, (colname, color) in enumerate(zip(projects_list, category_colors)):     
        widths = employee_project_dataframes[emp_names].loc[colname]
        print(widths)
        starts = employee_project_dataframes_cumsum[emp_names].loc[colname] - widths
        rects = ax.barh(labels, widths, left=starts,
                    label=colname, color=color)
        r, g, b, _ = color
        text_color = 'white' if r * g * b < 0.2 else 'black'
        # print rects
        rects.datavalues = [np.nan if i == 0 else i for i in rects.datavalues]
        # print(rects.datavalues)
        ax.bar_label(rects, labels=rects.datavalues, 
                     label_type='center', color=text_color, padding=1, fmt='%.1f')
    if(not first_one_done):
        ax.legend(ncol=len(projects_list), loc='upper center',
                    fontsize='large', bbox_to_anchor=(0.5, 2))
        first_one_done = True
    
# fig.suptitle('Employee Project Distribution')

In [ ]:
plt.style.use('default')

In [ ]:
import seaborn as sns
matplotx.styles.dufte


In [ ]:
# ax.set_color_cycle(sns.color_palette("coolwarm_r", num_lines))
my_colors = ListedColormap(sns.color_palette())


In [ ]:
sns.color_palette()

In [ ]:
from matplotlib.colors import ListedColormap


In [165]:
variables_df = pd.read_excel(
    os.path.join(os.getcwd(), "./Main.xlsx"), sheet_name="Variables", header=0,
)


In [166]:
variables_df

,Variable,Value
0,Number of People per Graph,4
1,Number of Working Hours Every Month,150


In [171]:
variables_df.iloc[:8]

,Variable,Value
0,Number of People per Graph,4
1,Number of Working Hours Every Month,150
